In [30]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('/home/jupyter-23521027/refresh-bert/data/sbert/indosbert_dense_trained', device='cuda:1')

In [31]:
import os
import json
import pandas as pd

folder_path = '/home/jupyter-23521027/refresh-bert/data/liputan6_data/canonical/train' 

file_list = os.listdir(folder_path)
json_data_list = []
file_list = os.listdir(folder_path)

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name) 

    if os.path.isfile(file_path):
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            json_data_list.append(json_data)

df = pd.DataFrame(json_data_list)

In [32]:
df

,id,url,clean_article,clean_summary,extractive_summary
0,48951,https://www.liputan6.com/news/read/48951/polis...,"[[Liputan6, ., com, ,, Lamongan, :, Tim, Inves...","[[Senjata, api, dan, ratusan, butir, peluru, m...","[1, 2]"
1,124973,https://www.liputan6.com/news/read/124973/gent...,"[[Liputan6, ., com, ,, Depok, :, Abdul, Madjid...","[[Genta, buatan, Abdul, Madjid, bisa, melahirk...","[0, 1]"
2,298677,https://www.liputan6.com/news/read/298677/wasp...,"[[Liputan6, ., com, ,, Bekasi, :, Aparat, Kepo...","[[Berhati-hatilah, Anda, bila, bepergian, ke, ...","[0, 5]"
3,40569,https://www.liputan6.com/news/read/40569/imbau...,"[[Liputan6, ., com, ,, Denpasar, :, Imbauan, M...","[[Peringatan, untuk, tidak, pergi, ke, Indones...","[5, 8]"
4,242869,https://www.liputan6.com/news/read/242869/rapa...,"[[Liputan6, ., com, ,, Jakarta, :, Gempa, kuat...","[[Rapat, pleno, KPU, terhenti, ketika, gempa, ...","[0, 4]"
...,...,...,...,...,...
193878,164143,https://www.liputan6.com/news/read/164143/tahu...,"[[Liputan6, ., com, ,, Jakarta, :, Warga, yang...","[[Dirjen, Pajak, membuat, aturan, setiap, oran...","[2, 4]"
193879,150247,https://www.liputan6.com/news/read/150247/mass...,"[[Liputan6, ., com, ,, Bogor, :, Ribuan, umat,...","[[Ribuan, umat, Islam, menyerbu, vila, milik, ...","[0, 1]"
193880,204203,https://www.liputan6.com/news/read/204203/bate...,"[[Chelsea, melayangkan, keberatannya, kepada, ...","[[Ketua, Leeds, United, ,, Ken, Bates, berhasi...","[0, 13]"
193881,159188,https://www.liputan6.com/news/read/159188/peny...,"[[Liputan6, ., com, ,, Denpasar, :, Kebakaran,...","[[Kebakaran, toko, sepatu, di, Denpasar, ,, Ba...","[0, 3]"


In [33]:
df['sentences'] = df['clean_article'].apply(lambda x: [' '.join(sentence) for sentence in x])

In [34]:
df.head(1)

,id,url,clean_article,clean_summary,extractive_summary,sentences
0,48951,https://www.liputan6.com/news/read/48951/polis...,"[[Liputan6, ., com, ,, Lamongan, :, Tim, Inves...","[[Senjata, api, dan, ratusan, butir, peluru, m...","[1, 2]","[Liputan6 . com , Lamongan : Tim Investigasi B..."


In [37]:
from datetime import datetime

MAX_DOC_LENGTH = 100
MAX_SENT_EMBEDDING_LENGTH = 250
DATASET_SIZE = 5000
docs = []

now = datetime.now()
file = open("/home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.training.sbert" , 'w')

for i, row in df.iterrows():
    file.write(f"liputan6-{str(row['id'])}\n")

    for sent_txt in row["sentences"][:MAX_DOC_LENGTH]:
        sent_embedding = model.encode(sent_txt, convert_to_numpy=True).tolist()[:MAX_SENT_EMBEDDING_LENGTH]

        for element in sent_embedding:
            file.write(str(element) + " ")
            
        file.write("\n")
    
    if (i + 1) % 100 == 0:
        print(f"processing {i + 1} with time: {datetime.now() - now}")
        now = datetime.now()
    
    if (i + 1) == DATASET_SIZE:
        break
    
    file.write("\n")
    
file.close()

processing 100 with time: 0:00:23.171049
processing 200 with time: 0:00:21.065867
processing 300 with time: 0:00:18.864559
processing 400 with time: 0:00:16.309201
processing 500 with time: 0:00:21.377194
processing 600 with time: 0:00:19.897849
processing 700 with time: 0:00:18.952877
processing 800 with time: 0:00:18.627153
processing 900 with time: 0:00:20.126858
processing 1000 with time: 0:00:18.712037
processing 1100 with time: 0:00:19.813273
processing 1200 with time: 0:00:20.447449
processing 1300 with time: 0:00:19.640685
processing 1400 with time: 0:00:18.475355
processing 1500 with time: 0:00:20.014538
processing 1600 with time: 0:00:21.022673
processing 1700 with time: 0:00:17.457113
processing 1800 with time: 0:00:19.348339
processing 1900 with time: 0:00:21.735999
processing 2000 with time: 0:00:20.988240
processing 2100 with time: 0:00:18.808061
processing 2200 with time: 0:00:18.860903
processing 2300 with time: 0:00:20.855935
processing 2400 with time: 0:00:20.156916
p

In [40]:
!cat /home/jupyter-23521027/refresh-bert/data/preprocessed-input-directory/liputan6.validation.sbert | head -c 1000

8279
-0.710845947265625 -0.6578059196472168 -0.24995559453964233 -0.7937917709350586 0.505650520324707 0.025467798113822937 -0.7826764583587646 -0.34205272793769836 -0.547825276851654 0.2897303104400635 -0.49375829100608826 -0.526947021484375 -0.47405552864074707 0.026531340554356575 0.47491583228111267 0.2732073962688446 0.8166688680648804 0.2826261818408966 0.032698068767786026 -0.4343765377998352 -0.19703309237957 -0.7972460985183716 -0.04794219508767128 -0.23798783123493195 -0.31992307305336 -0.34512844681739807 0.25353503227233887 0.7340987920761108 -0.4388556480407715 0.43969058990478516 0.07167457044124603 -0.761844277381897 0.08425012975931168 -0.12586335837841034 0.2394513636827469 -0.11480709165334702 -0.7919788956642151 0.29926925897598267 -0.6236393451690674 -0.08952662348747253 -0.3983997702598572 -0.5265697836875916 0.01723548211157322 -0.4459589421749115 -0.04457573965191841 0.2764419913291931 -0.26462045311927795 0.5729169845581055 -0.3529398739337921 0.0672292187809944